In [132]:
import numpy as np

In [133]:
# 逻辑电路设置，X输入向量， W为调节阈值，通过不同的调试可以实现 与或非 和 与非门
def logicalGate(X, W):
    return 1 if np.sum(W * X) > 0 else 0

In [134]:
# 与门 输入电位 减半叠加 和 0.75相比来判断是否为 高低电位
def AND(x1, x2):
    return logicalGate(np.array([1,x1,x2]), np.array([-0.75, 0.5, 0.5]))

In [135]:
# 或门
def OR(x1, x2):
    return logicalGate(np.array([1,x1,x2]), np.array([-0.25, 0.5, 0.5]))

In [136]:
# 非门
def NOT(x1):
    return logicalGate(np.array([1,x1,x1]), np.array([0.75, -0.5, -0.5]))

In [137]:
# 与非门
def NAND(x1, x2):
    return logicalGate(np.array([1,x1,x2]), np.array([0.75, -0.5, -0.5]))

In [138]:
# 异或门
def XOR(x1, x2):
    return AND(OR(x1,x2),NAND(x1,x2))

In [139]:
#测试单目运算
NOT(0),NOT(1)

(1, 0)

In [140]:
# 测试其他的逻辑运算
def testGate(gates) :
    ret = []
    for x1 in range(2):
        for x2 in range(2): 
            item = []
            for gate in gates :
                item.append(gate(x1,x2))
            ret.append(item)
    return ret

In [141]:
def printTestRet(ret):
    print ( 'x1\tx2\tand\tor\tnand\txor' )
    for x1 in range(2):
        for x2 in range(2):
            print(x1, '\t', x2, end ='\t')
            for i in range(4):
                print(ret[x1*2+x2][i], end = '\t' )
            print()

In [142]:
ret = testGate([AND,OR,NAND,XOR])
printTestRet(ret)

x1	x2	and	or	nand	xor
0 	 0	0	0	1	0	
0 	 1	0	1	1	1	
1 	 0	0	1	1	1	
1 	 1	1	1	0	0	


In [143]:
# 用与非门 实现 与门
def AND2(x1, x2):
    return NAND(NAND(x1,x2), NAND(x1,x2) )

In [144]:
# 用与非门 实现 或门
def OR2(x1, x2):
    return NAND(NAND(x1,x1), NAND(x2,x2) )

In [145]:
# 用与非门 实现 非门
def NOT2(x1):
    return NAND(x1,x1)

In [146]:
# 用与非门 实现 与非门 ，就是本身
def NAND2(x1, x2):
    return NAND(x1, x2 )

In [147]:
# 用与非门 实现 异或门
def XOR2(x1, x2):
    return NAND(NAND(x1, NAND(x2,x2) ), NAND(NAND(x1,x1),x2) )

In [148]:
NOT2(0),NOT2(1)

(1, 0)

In [149]:
ret2 = testGate([AND2,OR2,NAND2,XOR2])
printTestRet(ret2)

x1	x2	and	or	nand	xor
0 	 0	0	0	1	0	
0 	 1	0	1	1	1	
1 	 0	0	1	1	1	
1 	 1	1	1	0	0	


In [150]:
def XOR1(x1, x2):
    return AND(OR(x1,x2),NOT(AND(x1,x2)))

In [151]:
for xs in [(0, 0), (1, 0), (0, 1), (1, 1)]:
    y = XOR1(xs[0], xs[1])
    print(str(xs) + " -> " + str(y))

(0, 0) -> 0
(1, 0) -> 1
(0, 1) -> 1
(1, 1) -> 0


In [152]:
# 半加器的实现
def halfAdd(x1, x2):
    return (XOR(x1, x2), AND(x1, x2))

In [153]:
# 全加器，只能实现一位二进制相加， carryBit 进位
def fullAdd(x1, x2, carryBit):
    s, co = halfAdd(x1, x2)
    s, co2 = halfAdd(carryBit, s)
    return (s, OR(co, co2))

In [154]:
for xs in [(0, 0, 0), (1, 0, 0), (0, 1, 0), (1, 1, 0), (0, 0, 1), (1, 0, 1), (0, 1, 1), (1, 1, 1)]:
    y = fullAdd(xs[0], xs[1], xs[2])
    print(str(xs) + " -> " + str(y))

(0, 0, 0) -> (0, 0)
(1, 0, 0) -> (1, 0)
(0, 1, 0) -> (1, 0)
(1, 1, 0) -> (0, 1)
(0, 0, 1) -> (1, 0)
(1, 0, 1) -> (0, 1)
(0, 1, 1) -> (0, 1)
(1, 1, 1) -> (1, 1)


In [155]:
# 16位二进制相加， co 为是否进位标志
def add16Bit(a16bit, b16bit, co=0):
    v =[]
    for i in range(16):
        (d, co) = fullAdd(a16bit[i], b16bit[i], co)
        v.append(d % 2)
    return v

In [156]:
# 辅助方法，用数组来表示 16位数
def to16bit(x):
    v16bit =[]
    d = x
    for i in range(16):
        v16bit.append(d % 2)
        d = d // 2
    return v16bit

In [157]:
# 辅助方法，将16位数数组 转换为整数
def from16bit(a16bit):
    d=0
    for i in range(15,-1,-1):
        d = d * 2 + a16bit[i]
    return d
        

In [158]:
# 加法的实现
def add(a, b):
    return from16bit(add16Bit(to16bit(a), to16bit(b)))

In [159]:
add(6,7)

13

In [160]:
# 16位数的补码，就是每一位取反， 但是最低为没有+1
def complement(a16bit):
    for i in range(16):
        a16bit[i] = NOT(a16bit[i])
    return a16bit

In [161]:
# 16位数的减法实现
def sub16Bit(a16bit, b16bit): 
    return add16Bit(a16bit, complement(b16bit), 1)

In [162]:
def sub(a, b):
    return from16bit(sub16Bit(to16bit(a), to16bit(b)))

In [163]:
sub(20, 8)

12